# Includes

In [ ]:
import sys
import csv
def find_and_set_sys_maxsize():
    maxInt = sys.maxsize
    decrement = True
    while decrement:
        decrement = False
        try:
            csv.field_size_limit(maxInt)
        except OverflowError:
            maxInt = int(maxInt/10)
            decrement = True
find_and_set_sys_maxsize()

import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)

import pickle
import datetime
def timestamp(): return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
def print_progress(idx, final_idx, min_step = 1):
    min_step = min_step if min_step <= final_idx else final_idx
    if idx % (final_idx // 100 * min_step) == 0: print("%.0f%% done" % (idx / final_idx * 100), end='\r')
    
import os
import math
import pandas as pd
import numpy as np

import gensim
import nltk
nltk.download('punkt')

from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.word2vec import Word2Vec
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

from scipy.sparse import hstack
from scipy.sparse import save_npz
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB as NB

In [2]:
results_output_folder = './output/'

# Reading dataset

In [3]:
granddebat_folder = '../GrandDébat/granddebat.fr/'
democrat_path = os.path.join(granddebat_folder, 'DEMOCRATIE_ET_CITOYENNETE.csv')
events_path = os.path.join(granddebat_folder, 'EVENTS.csv')
fiscalit_path = os.path.join(granddebat_folder, 'LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.csv')
ecolog_path = os.path.join(granddebat_folder, 'LA_TRANSITION_ECOLOGIQUE.csv')
organisat_path = os.path.join(granddebat_folder, 'ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.csv')

french_encoding = "utf-8"

In [4]:
file_path = democrat_path

In [5]:
db_raw = pd.read_csv(file_path, engine='python', encoding=french_encoding)

# Preparing dataframe

In [6]:
questions_reverse_index = 37
db_questions = db_raw.loc[:, db_raw.columns[-questions_reverse_index:]]

question_list = db_raw.columns[-questions_reverse_index:]
question_list = [el.split()[0][-2:] for el in question_list]
db_questions.columns = question_list

db_questions.head()

,A3,A4,A5,Ew,Ex,Ey,Ez,E0,E1,E2,...,M1,M2,M3,M4,M5,Qx,Qy,Qz,Q0,Q1
0,Le citoyen,Non,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afin d’éviter de creuser les inégalités ne plu...
1,Un instrument de démocratie locale à modernise...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nous proposons le retour à la limitation de vi...
3,Voir l'intégralité de la proposition dans la d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POUR UN NOUVEAU CONTRAT CITOYEN ...
4,"député, maire, moi même",Non,NaN,"Budget participatif, possibilité d'interpeller...",Une bonne chose,NaN,NaN,Oui,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Getting raw X and y

In [7]:
def preprocess_text(text):
    text = " ".join(gensim.utils.simple_preprocess(text))
    return text

In [8]:
%%time

db = db_questions

texts = []
authorZipCodes = []

question_numbers = []

for row in range(db.shape[0]):
    print_progress(row, db.shape[0], 1)
    for col in range(db.shape[1]):
        if not pd.isnull(db.iat[row, col]):
            text = preprocess_text(db.iat[row, col])
            authorZipCode = db_raw.at[row, 'authorZipCode']
            question_number = col

            texts.append(text)
            authorZipCodes.append(authorZipCode)
            
            question_numbers.append(question_number)
authorZipCodes_np = np.array(authorZipCodes, dtype=np.int64)

Wall time: 2min 21s


In [ ]:
%%time

db = db_questions

texts = []
authorZipCodes = []

for row in range(db.shape[0]):
    print_progress(row, db.shape[0], 1)
    
    author_texts = []
    authorZipCode = db_raw.at[row, 'authorZipCode']
    authorZipCodes.append(authorZipCode)
    for col in range(db.shape[1]):
        if not pd.isnull(db.iat[row, col]):
            text = preprocess_text(db.iat[row, col])
            author_texts.append(text)
    texts.append(" ".join(author_texts))
            
            
authorZipCodes_np = np.array(authorZipCodes, dtype=np.int64)

In [47]:
np.savetxt(os.path.join(results_output_folder, 'y_authorZipCodes.csv'), authorZipCodes_np)

# Preprocessing y

## Postal code regions (~100 of them)

In [20]:
def get_ZipCode_postal_region(zipCode):
    return int(str(zipCode)[0:2])

In [48]:
authorZipCode_postal_regions = []

In [49]:
%%time

for ZipCode in authorZipCodes:
    authorZipCode_postal_region = get_ZipCode_postal_region(ZipCode)
    authorZipCode_postal_regions.append(authorZipCode_postal_region)

authorZipCode_postal_regions = np.array(authorZipCode_postal_regions, dtype=np.int32)

Wall time: 2.77 s


In [55]:
np.savetxt(os.path.join(results_output_folder, 'y_authorZipCode_postal_regions.csv'), authorZipCode_postal_regions)

## Geo regions (14 of them)

In [17]:
region_names_to_zipCode_regions = {
"Île de France": ["75", "77", "78", "91", "92", "93", "94", "95"],
"Nord-Pas-de-Calais Picardie": ["02", "59", "60", "62", "80"],
"Alsace Champagne-Ardenne Lorraine": ["08", "10", "51", "52", "54", "55", "57", "67", "88", "68"],
"Normandie": ["14", "27", "50", "61", "76"],
"Bretagne": ["22", "29", "35", "56"],
"Pays de la Loire": ["44", "49", "53", "72", "85"],
"Centre Val de Loire": ["18", "28", "36", "37", "41", "45"],
"Bourgogne Franche-Comté": ["21", "25", "39", "58", "70", "71", "89", "90"],
"Aquitaine Limousin Poitou-Charentes": ["16", "17", "19", "23", "24", "33", "40", "47", "64", "79", "86", "87"],
"Auvergne-Rhône-Alpes": ["01", "03", "07", "15", "26", "38", "42", "43", "63", "69", "73", "74"],
"Languedoc-Roussillon Midi Pyrénées": ["09", "11", "12", "30", "31", "32", "34", "46", "48", "65", "66", "81", "82"],
"Provence Alpes Côte d'Azur": ["04", "05", "06", "13", "83", "84"],
"Corse": ["20"],
}

In [18]:
def strip_zipCode(zipCode):
    return str(zipCode)[0:2]

def get_region_by_stripped_zipCode(zipCode):
    region_index = None
    for idx, region in enumerate(region_names_to_zipCode_regions):
        if zipCode in region_names_to_zipCode_regions[region]:
            region_index = idx
    return region_index

def get_geo_region_by_ZipCode(zipCode):
    zipCode = strip_zipCode(zipCode)
    return get_region_by_stripped_zipCode(zipCode)

In [19]:
authorZipCode_geo_regions = []

In [20]:
%%time

invalid_zipcodes = []
for idx, ZipCode in enumerate(authorZipCodes):
    print_progress(idx, len(authorZipCodes))
    authorZipCode_geo_region = get_geo_region_by_ZipCode(ZipCode)
    
    if authorZipCode_geo_region == None:
        authorZipCode_geo_region = -1
        invalid_zipcodes.append(strip_zipCode(ZipCode))
        
    authorZipCode_geo_regions.append(authorZipCode_geo_region)

authorZipCode_geo_regions = np.array(authorZipCode_geo_regions, dtype=np.int32)

invalid_zipcodes = set(invalid_zipcodes)
print("zipCodes that does not belong to any region:", invalid_zipcodes)

zipCodes that does not belong to any region: {'9', '97', '3', '1', '99', '5', '98', '6', '7', '0', '96', '4'}
Wall time: 10.7 s


In [67]:
np.savetxt(os.path.join(results_output_folder, 'y_authorZipCode_geo_regions.csv'), authorZipCode_geo_regions)

## If city is big or not {0, 1}

In [10]:
cities_data_folder = '../data_base/'
cities_data_path = os.path.join(cities_data_folder, 'correspondance-code-insee-code-postal.csv')

In [11]:
cities_data = pd.read_csv('../data_base/correspondance-code-insee-code-postal.csv', ";")

`density_thresh` is a threshold to consider between big city and small city in terms of density of population

In [12]:
french_density = 117.48
density_thresh = french_density * 5

### Initial

The code in this subregion is taken from the Data_set.py script

In [89]:
def get_density_by_zip(zipCode, cities_data):
    df = cities_data[cities_data['Code Postal'] == str(zipCode)]
    
    density = (df['Population'].sum() * 1000) / (df['Superficie'].sum() * 0.01)
    if np.isnan(density):
        density = 0
        
    return density

def is_city_large(density, density_thresh):
    return int(density > density_thresh)

In [90]:
city_binary_sizes = []

In [ ]:
%%time

for idx, ZipCode in enumerate(authorZipCodes):
    print_progress(idx, len(authorZipCodes))
    if_city_is_large = is_city_large(get_density_by_zip(ZipCode, cities_data), density_thresh)
    city_binary_sizes.append(if_city_is_large)
city_binary_sizes = np.array(city_binary_sizes)

### Adaptation

In [14]:
%%time

cities_data_cache = {}
for idx, ZipCode in enumerate(authorZipCodes):
    print_progress(idx, len(authorZipCodes))
    if not ZipCode in cities_data_cache:       
        df = cities_data[cities_data['Code Postal'] == str(ZipCode)]
        density = (df['Population'].sum() * 1000) / (df['Superficie'].sum() * 0.01)
        cities_data_cache[ZipCode] = density

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


Wall time: 31.6 s


In [15]:
def get_density_by_zip(zipCode, cities_data_cache):
    return cities_data_cache[zipCode]

def is_city_large(density, density_thresh):
    return int(density > density_thresh)

In [61]:
city_binary_sizes = []

In [62]:
%%time

for idx, ZipCode in enumerate(authorZipCodes):
    print_progress(idx, len(authorZipCodes))
    if_city_is_large = is_city_large(get_density_by_zip(ZipCode, cities_data_cache), density_thresh)
    city_binary_sizes.append(if_city_is_large)
city_binary_sizes = np.array(city_binary_sizes, dtype=np.int16)

Wall time: 4.6 s


In [63]:
np.savetxt(os.path.join(results_output_folder, 'y_city_binary_sizes.csv'), city_binary_sizes)

# X from simple NLP models application

## TF-IDF

In [30]:
X_tfidf = None

In [31]:
%%time

vectorizer = TfidfVectorizer(min_df = 1)
X_tfidf = vectorizer.fit_transform(texts)

In [32]:
save_npz(os.path.join(results_output_folder, 'X_tfidf.npz'), X_tfidf)

You can do load_npz by `from scipy.sparse import load_npz`

## Word2Vec + PCA

### Word2Vec

https://github.com/Kyubyong/wordvectors  
French (w)

In [9]:
w2vec_pretrained_folder = './pretrained_w2v/'
w2vec_pretrained_path = os.path.join(w2vec_pretrained_folder, 'fr.bin')

In [10]:
w2v_model = Word2Vec.load(w2vec_pretrained_path)

In [11]:
w2vec_texts_vectors = []

In [12]:
%%time

unknown_to_pretrained_word_list = []
dtype = np.float32
for idx, text in enumerate(texts):
    print_progress(idx, len(texts))
    
    word_vectors = []
    for word in text.split():
        try:
            word_vectors.append(w2v_model[word])
        except:
            if not word in unknown_to_pretrained_word_list:
                unknown_to_pretrained_word_list.append(word)

    if not len(word_vectors) == 0:
        word_vectors = np.array(word_vectors, dtype = dtype)
        words_mean_vector = word_vectors.sum(axis = 0) / word_vectors.shape[0]
    else:
        words_mean_vector = np.zeros((300,), dtype = dtype)
        
    w2vec_texts_vectors.append(words_mean_vector)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


Wall time: 11min 37s


In [13]:
w2vec_texts_vectors_np = np.vstack(w2vec_texts_vectors)

In [26]:
# w2vec_texts_vectors_np = np.array(w2vec_texts_vectors, dtype=np.float16)

In [125]:
# w2vec_texts_vectors_np -= w2vec_texts_vectors_np.min()
# w2vec_texts_vectors_np /= w2vec_texts_vectors_np.max()

### PCA

You will probably get a MemoryError if you haven't already

In [28]:
n_components = 30

In [ ]:
%%time

pca = PCA(n_components = n_components)
X_w2vec_pca = pca.fit_transform(w2vec_texts_vectors_np) # w2vec_texts_vectors

In [134]:
filename = 'X_w2vec_pca_%d_components.csv' % (n_components)
np.savetxt(os.path.join(results_output_folder, filename), X_w2vec_pca, delimiter = ',')

In [ ]:
del pca

### Truncated SVD

This can perform somewhat similar but without MemoryError

In [14]:
n_components = 30

In [15]:
%%time

svd = TruncatedSVD(n_components=n_components, algorithm='arpack')
X_w2vec_svd = svd.fit_transform(w2vec_texts_vectors_np)

In [16]:
filename = 'X_w2vec_svd_%d_components.csv' % (n_components)
np.savetxt(os.path.join(results_output_folder, filename), X_w2vec_svd, delimiter = ',')

# Model check

In [135]:
def get_model_accuracy(model, X_test, y_test, y):
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]

    accuracy = accuracy_score(y_test, predictions)
    print("Model accuracy: %.2f%%" % (accuracy * 100.0))
    random_accuracy = 1 / len(set(y))
    print("Random accuracy: %.2f%%" % (random_accuracy * 100.0))

## Selecting data

In [136]:
X = X_tfidf
y = authorZipCode_geo_regions

In [137]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

## XGBoost

In [144]:
dtrain = xgb.DMatrix(X_train, label = y_train)

In [ ]:
%%time

param = {}
param['nthread'] = 4
param['max_depth'] = 6
num_boost_round = 10

xgb_model = xgb.train(param, dtrain, num_boost_round = num_boost_round)

In [151]:
xgb_X_test = xgb.DMatrix(X_test)
get_model_accuracy(xgb_model, xgb_X_test, y_test, y)

Model accuracy: 5.62%
Random accuracy: 7.14%


## Multinomial Naive Bayes

`fit_prior = False` decreases acc by 0.1% which means that we have a slight imbalance between zipcodes

In [138]:
%%time

nb_model = NB().fit(X = X_train, y = y_train)

In [143]:
get_model_accuracy(nb_model, X_test, y_test, y)

Model accuracy: 21.43%
Random accuracy: 7.14%
